# Algoritmos

## Imports


In [ ]:
!pip install tqdm
from tqdm import tqdm

## Pacientes

In [2]:
columns_aih = ['id_paciente', 'co_cid_principal', 'no_cid_principal', 'co_procedimento_principal',
               'co_cid_secundario', 'no_cid_secundario', 'procedimento_principal', 'desc_procedimento_secundario']

columns_apac = ['id_paciente', 'co_cid_principal', 'no_cid_principal', 'co_cid_secundario',
              'co_procedimento_principal', 'no_procedimento_principal', 'co_procedimento_secundario', 'no_procedimento_secundario']

columns_bpai = ['id_paciente', 'co_cid_principal', 'no_cid_principal', 'co_procedimento_realizado', 'no_procedimento_realizado', ]

In [5]:
import pandas as pd
path = 'data/HD/'
pac_apac = pd.read_parquet(path+'APAC_Lupus_L93M32N08_todos_cids.parquet', columns=columns_apac)
pac_aih = pd.read_parquet(path+'AIH_Lupus_L93M32N08_todos_cids.parquet', columns=columns_aih)
pac_bpai = pd.read_parquet(path+'BPAI_Lupus_L93M32N08_todos_cids.parquet', columns=columns_bpai)

FileNotFoundError: [Errno 2] No such file or directory: 'data/HD/APAC_Lupus_L93M32N08_todos_cids.parquet'

In [138]:
print(f' - APAC: Registros: {pac_apac.shape[0]} Pacientes únicos: {pac_apac['id_paciente'].nunique()}')
print(f' - AIH: Registros: {pac_aih.shape[0]} Pacientes únicos: {pac_aih['id_paciente'].nunique()}')
print(f' - BPAI: Registros: {pac_bpai.shape[0]} Pacientes únicos: {pac_bpai['id_paciente'].nunique()}')
print(f' - Total: Registros: {pac_bpai.shape[0]+pac_aih.shape[0]+pac_apac.shape[0]} Pacientes únicos: {pac_bpai['id_paciente'].nunique()+pac_aih['id_paciente'].nunique()+pac_apac['id_paciente'].nunique()}')


 - APAC: Registros: 7194178 Pacientes únicos: 31948
 - AIH: Registros: 3968946 Pacientes únicos: 48663
 - BPAI: Registros: 857369 Pacientes únicos: 46967
 - Total: Registros: 12020493 Pacientes únicos: 127578


In [5]:
set_bpai = set(pac_bpai['id_paciente'])
set_apac = set(pac_apac['id_paciente'])
set_aih = set(pac_aih['id_paciente'])

print(' - Interseção do aih com apac:', len(set_aih.intersection(set_apac)))
print(' - Interseção do aih com bpai:', len(set_bpai.intersection(set_aih)))
print(' - Interseção do bpai com apac:', len(set_apac.intersection(set_bpai)))
print(' - Interseção dos 3 conjuntos:', len(set_apac.intersection(set_bpai).intersection(set_aih)))

 - Interseção do aih com apac: 28856
 - Interseção do aih com bpai: 25601
 - Interseção do bpai com apac: 19485
 - Interseção dos 3 conjuntos: 18238


## Filtrando CIDs

In [186]:
def verifica_cid(row):
    for cod in ['M32', 'L93', 'N08']:
        if (cod in row):
            return True    
    return False

for df in tqdm([pac_aih, pac_bpai, pac_apac]):
    df['filtro_cid_principal'] = df['co_cid_principal'].apply(verifica_cid)
    if 'co_cid_secundario' in df.columns:
        df['filtro_cid_secundario'] = df['co_cid_secundario'].apply(verifica_cid)

100%|██████████| 3/3 [00:07<00:00,  2.58s/it]


In [187]:
print(' - AIH')
print(pac_aih['filtro_cid_principal'].value_counts(normalize=True))
print()
print(pac_aih['filtro_cid_secundario'].value_counts(normalize=True))

print()
print(' - APAC')
print(pac_apac['filtro_cid_principal'].value_counts(normalize=True))
print()
print(pac_apac['filtro_cid_secundario'].value_counts(normalize=True))
print()
print(' - BPAI')
print(pac_bpai['filtro_cid_principal'].value_counts(normalize=True))

 - AIH
filtro_cid_principal
False    0.774591
True     0.225409
Name: proportion, dtype: float64

filtro_cid_secundario
False    0.948285
True     0.051715
Name: proportion, dtype: float64

 - APAC
filtro_cid_principal
False    0.984727
True     0.015273
Name: proportion, dtype: float64

filtro_cid_secundario
False    0.802416
True     0.197584
Name: proportion, dtype: float64

 - BPAI
filtro_cid_principal
False    0.922196
True     0.077804
Name: proportion, dtype: float64


## Classificando os Pacientes

In [67]:
import importlib
from  src import procedimentos
importlib.reload(procedimentos)

def verifica_procedimentos(df_pacientes, col_procedimento):
    df_pacientes = df_pacientes.copy()
    pacientes_list = list(df_pacientes.id_paciente.unique())
    dict_pacientes = {}
    proc_dict = {'p1':procedimentos.p1, 'p2':procedimentos.p2, 'p3':procedimentos.p3, 'p4':procedimentos.p4}
    
    for proc_name in proc_dict.keys():
        dict_pacientes[proc_name] = set()

    for id_paciente in tqdm(pacientes_list, desc=f'Classificando Pacientes'):
        id_paciente = int(id_paciente)
        df_paciente = df_pacientes[df_pacientes.id_paciente==id_paciente].copy() # Pega apenas os procedimentos no paciente em questão        

        # Se o paciente passou no filtro dos cids, agora aplica o filtro dos procedimentos
        for proc, proc_func in proc_dict.items():
            if proc_func(df_paciente, col_procedimento): # Aplica todos os procedimentos em cada cliente
                dict_pacientes[proc].add(id_paciente) # Salva o id do paciente no conjunto do procedimento em que ele passou

    for proced in ['p1', 'p2', 'p3', 'p4']: # Adiciona uma coluna como True para cada procedimento em que o paciente foi aprovado
        df_pacientes[f'procedimento_{proced}'] = df_pacientes['id_paciente'].isin(dict_pacientes[proced])
    
    return df_pacientes

In [ ]:
df_apac = verifica_procedimentos(pac_apac, col_procedimento='no_procedimento_principal')
df_aih = verifica_procedimentos(pac_aih, col_procedimento='desc_procedimento_secundario')
df_bpai = verifica_procedimentos(pac_bpai, col_procedimento='no_procedimento_realizado')

Classificando Pacientes: 100%|██████████| 48663/48663 [05:00<00:00, 162.08it/s]


### Aplicando Algoritmos

In [ ]:
# Juntando as bases (por enquanto BPAI, APAC e AIH)
cols_union = ['id_paciente', 'procedimento_p1', 'procedimento_p2', 'procedimento_p3', 'procedimento_p4', 'filtro_cid_principal']
df_union = pd.concat([df_bpai[cols_union], df_apac[cols_union+['filtro_cid_secundario']], df_aih[cols_union+['filtro_cid_secundario']]])
df_union.fillna(False, inplace=True)

print('Número de registros')
print(f' - AIH: {df_aih.shape[0]}')
print(f' - BPAI: {df_bpai.shape[0]}')
print(f' - APAC: {df_apac.shape[0]}')
print()
print('Número de Pacientes Únicos:')
print(f' - AIH: {df_aih['id_paciente'].nunique()}')
print(f' - BPAI: {df_bpai['id_paciente'].nunique()}')
print(f' - APAC: {df_apac['id_paciente'].nunique()}')

Número de registros
 - AIH: 3968946
 - BPAI: 857369
 - APAC: 7194178

Número de Pacientes Únicos:
 - AIH: 48663
 - BPAI: 46967
 - APAC: 31948


In [243]:
print(' - Tamanho da união dos conjuntos:', df_union.shape)

(12020493, 7)

In [194]:
df_cid = df_union[df_union['filtro_cid_principal'] | df_union['filtro_cid_secundario']].copy()
print('Número de Registros:')
print(' - Aplicando o filtro do cid')
print(f' - Redução de {df_union.shape[0]} para {df_cid.shape[0]} = {round((df_union.shape[0]-df_cid.shape[0])*100/df_union.shape[0],2)}%')

print()
num_pac_union = df_union['id_paciente'].nunique()
num_pac_cid = df_cid['id_paciente'].nunique()
print('Número de Pacientes Únicos:')
print(f' - Antes do filtro: {num_pac_union}')
print(f' - Após o filtro: {num_pac_cid}')
red_pac = (num_pac_union-num_pac_cid)/num_pac_union
print(f' - Redução de {round(red_pac*100,2)}%')
print(f' - Ou seja, {round(num_pac_cid*100/num_pac_union,2)}% aprovados no filtro do cid')

Número de Registros:
 - Aplicando o filtro do cid
 - Redução de 12020493 para 2657726 = 77.89%

Número de Pacientes Únicos:
 - Antes do filtro: 71874
 - Após o filtro: 69772
 - Redução de 2.92%
 - Ou seja, 97.08% aprovados no filtro do cid


In [195]:
# Agora aplicando os filtros de procedimento
cols_proced = ['procedimento_p1', 'procedimento_p2', 'procedimento_p3', 'procedimento_p4']
df_cid_pac = df_cid[df_cid['procedimento_p1'] | df_cid['procedimento_p2'] | df_cid['procedimento_p3'] | df_cid['procedimento_p4']].groupby('id_paciente')[cols_proced].sum()

df_cid_filtro_proced = df_cid_pac.map(lambda row: 1 if row > 0 else 0)
df_final = df_cid_filtro_proced.sum(axis=1).sort_values()

### Algoritmo 1
CID + 2 Procedimentos distindos

In [213]:
df_alg1 = df_final[df_final > 1].to_frame().reset_index()
df_alg1.columns = ['id_paciente', 'num_procedimentos_qualificados']
print(' - Total de pacientes encontrados após o algoritmo 1:', df_alg1.shape[0])
print(f' - % em relação ao total de pacientes (BPAI+APAC+AIH): {round((df_alg1.shape[0]*100)/df_union['id_paciente'].nunique(),2)}%')

 - Total de pacientes encontrados após o algoritmo 1: 10884
 - % em relação ao total de pacientes (BPAI+APAC+AIH): 15.14%


### Algoritmo 2
CID + 1 Procedimento

In [214]:
df_alg2 = df_final[df_final > 0].to_frame().reset_index()
df_alg2.columns = ['id_paciente', 'num_procedimentos_qualificados']
print(' - Total de pacientes encontrados após o algoritmo 2:', df_alg2.shape[0])
print(f' - % em relação ao total de pacientes (BPAI+APAC+AIH): {round((df_alg2.shape[0]*100)/df_union['id_paciente'].nunique(),2)}%')

 - Total de pacientes encontrados após o algoritmo 2: 23137
 - % em relação ao total de pacientes (BPAI+APAC+AIH): 32.19%


In [147]:
df_bpai.co_cid_principal.value_counts(normalize=True, dropna=False).iloc[:20]

co_cid_principal
        0.368758
        0.158768
Z006    0.027869
M321    0.023031
M329    0.020937
N180    0.012596
Z000    0.012543
M320    0.009392
Z017    0.008476
L930    0.007362
M328    0.007257
M545    0.007053
Z010    0.006820
M255    0.006297
Z048    0.006046
B182    0.005028
Z524    0.004671
I64     0.003771
Z940    0.003659
N083    0.003335
Name: proportion, dtype: float64

### Salvando os Resultados

In [215]:
df_alg1.to_parquet('data/results/df_alg1.parquet')
df_alg2.to_parquet('data/results/df_alg2.parquet')

In [217]:
df_alg1.id_paciente.nunique()/df_alg1.shape[0]

1.0

In [223]:
df_bpai_alg1 = df_bpai[df_bpai['id_paciente'].isin(df_alg1['id_paciente'])]
df_aih_alg1 = df_aih[df_aih['id_paciente'].isin(df_alg1['id_paciente'])]
df_apac_alg1 = df_apac[df_apac['id_paciente'].isin(df_alg1['id_paciente'])]

In [239]:
df_bpai_alg2 = df_bpai[df_bpai['id_paciente'].isin(df_alg2['id_paciente'])]
df_aih_alg2 = df_aih[df_aih['id_paciente'].isin(df_alg2['id_paciente'])]
df_apac_alg2 = df_apac[df_apac['id_paciente'].isin(df_alg2['id_paciente'])]

In [241]:
df_bpai_alg1.to_parquet('data/results/df_bpai_alg1.parquet')
df_aih_alg1.to_parquet('data/results/df_aih_alg1.parquet')
df_apac_alg1.to_parquet('data/results/df_apac_alg1.parquet')

df_bpai_alg2.to_parquet('data/results/df_bpai_alg2.parquet')
df_aih_alg2.to_parquet('data/results/df_aih_alg2.parquet')
df_apac_alg2.to_parquet('data/results/df_apac_alg2.parquet')

In [225]:
pacientes_alg1 = set(df_alg1['id_paciente'])

In [235]:
print(df_bpai_alg1.shape)
print(df_aih_alg1.shape)
print(df_apac_alg1.shape)

(245704, 10)
(1691355, 14)
(767139, 14)


In [238]:
print(' - Pacientes presentes no bpai:', df_bpai_alg1['id_paciente'].nunique() / df_alg1['id_paciente'].nunique())
print(' - Pacientes presentes no aih:', df_aih_alg1['id_paciente'].nunique() / df_alg1['id_paciente'].nunique())
print(' - Pacientes presentes no apac:', df_apac_alg1['id_paciente'].nunique() / df_alg1['id_paciente'].nunique())

 - Pacientes presentes no bpai: 0.6481991914737228
 - Pacientes presentes no aih: 0.9609518559353178
 - Pacientes presentes no apac: 0.6841234840132304
